### 说明
pytorch初步熟悉，参考https://github.com/fastai/fastai_docs/tree/master/dev_nb

### pathlib使用

In [ ]:
from pathlib import Path

# 打印路径下所有文件
print([x for x in Path('.').iterdir() if x.is_file()])

# 遍历查找python文件
print(list(Path('.').glob('**/*.py')))

# 创建文件夹
path = Path('data')/'mnist'
path.mkdir(parents=True, exist_ok=True)

### 下载数据

In [ ]:
import requests
url='http://deeplearning.net/data/mnist/'
filename='mnist.pkl.gz'

if not (path/filename).exists():
    content = requests.get(url+filename).content
    (path/filename).open('wb').write(content)

### 读取数据

In [ ]:
%matplotlib inline
from matplotlib import pyplot
import pickle, gzip
with gzip.open(path/filename, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

pyplot.imshow(x_train[0].reshape((28,28)), cmap="gray")